# Build a Conditional GAN

### Goals
In this notebook, you're going to make a conditional GAN in order to generate hand-written images of digits, conditioned on the digit to be generated (the class vector). This will let you choose what digit you want to generate.

You'll then do some exploration of the generated images to visualize what the noise and class vectors mean.  

### Learning Objectives
1.   Learn the technical difference between a conditional and unconditional GAN.
2.   Understand the distinction between the class and noise vector in a conditional GAN.



### Goals
In this notebook, you're going to make a conditional GAN in order to generate hand-written images of digits, conditioned on the digit to be generated (the class vector). This will let you choose what digit you want to generate.

You'll then do some exploration of the generated images to visualize what the noise and class vectors mean.  

### Learning Objectives
1.   Learn the technical difference between a conditional and unconditional GAN.
2.   Understand the distinction between the class and noise vector in a conditional GAN.



In [2]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader

torch.manual_seed(0)

In [3]:
def show_tensor_images(image_tensor,
                       num_images=25,
                       size=(1, 28, 28),
                       nrow=5,
                       show=True):
    """_summary_

    Args:
        image_tensor (_type_): _description_
        num_images (int, optional): _description_. Defaults to 25.
        size (tuple, optional): _description_. Defaults to (1, 28, 28).
        nrow (int, optional): _description_. Defaults to 5.
        show (bool, optional): _description_. Defaults to True.
    """
    image_tensor = (image_tensor + 1) / 2
    image_unflat = image_tensor.detach().cpu()
    image_grid = make_grid(image_unflat[:num_images], nrow=nrow)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    
    if show:
        plt.show()

### Generator and Noise

In [7]:
class Generator(nn.Module):
    def __init__(self, input_dim=10, im_chan=1, hidden_dim=64):
        super(Generator, self).__init__()
        self._input_dim = input_dim
        self._generator = nn.Sequential(
            self._make_gen_block(input_dim, hidden_dim * 4),
            self._make_gen_block(hidden_dim * 4, hidden_dim * 2, kernel_size=4, stride=1),
            self._make_gen_block(hidden_dim * 2, hidden_dim),
            self._make_gen_block(hidden_dim, im_chan, kernel_size=4, final_layer=True)
        )

    def _make_gen_block(self, input_channels, output_channels, kernel_size, stride, final_layer=False):
        if not final_layer:
            return nn.Sequential(
                nn.ConvTranspose2d(input_channels, output_channels, kernel_size, stride),
                nn.BatchNorm2d(output_channels),
                nn.ReLU(inplace=True)
            )
        else:
            return nn.Sequential(
                nn.ConvTranspose2d(input_channels, output_channels, kernel_size, stride),
                nn.Tanh()
            )
    
    def call(self, noise):
        x = noise.view(len(noise), self._input_dim, 1, 1)
        return self._generator(x)

def get_noise(n_samples, input_dim, device='cpu'):
    return torch.randn(n_samples, intput_dim, device=device)


import